# Noisy QCNN Kmnist

In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/home/tak/Github/QEmbedding/')
import torch
from torch import nn
import pennylane as qml
import embedding
import data
from qiskit.providers.fake_provider import FakeGuadalupe
from qiskit_aer.noise import NoiseModel


noisy = FakeGuadalupe()
noise_model = NoiseModel.from_backend(noisy)
coupling_map = noisy.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev_fake = qml.device(
    'qiskit.aer',
    wires=8,
    shots=1028,
    noise_model=noise_model,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

/tmp/ipykernel_2449848/110338151.py:14: DeprecationWarning: All fake backend instances based on real device snapshots (`FakeVigo`,`FakeSherbrooke`,...) have been migrated to the `qiskit_ibm_runtime` package. These classes are deprecated as of qiskit 0.46.0 and will be removed in qiskit 1.0.0. To migrate your code, run `pip install qiskit-ibm-runtime` and use `from qiskit_ibm_runtime.fake_provider import FakeExample` instead of `from qiskit.providers.fake_provider import FakeExample`. If you are using a custom fake backend implementation, you don't need to take any action.
  noisy = FakeGuadalupe()


# QCNN Training

In [2]:
batch_size = 15
steps = 200
ansatz = "TTN"
learning_rate = 0.05

In [3]:

class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(8, 24),
            nn.ReLU(),
            nn.Linear(24, 24),
            nn.ReLU(),
            nn.Linear(24, 15)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


model2 = x_transform2()

In [4]:
def statepreparation(x, data):
    if data == 'mnist':
        model2.load_state_dict(torch.load('NQE200_mnist.pt'))
    elif data == 'fashion':
        model2.load_state_dict(torch.load('NQE200_fashion.pt'))
    x = model2(torch.tensor(x))
    embedding.QuantumEmbedding2(x)

def data_reuploading_ansatz(params): #15 params
    for i in range(8):
        qml.RY(params[i], wires=i)
    for i in range(7):
        qml.IsingYY(params[i+8], wires=[i,i+1])

def data_reuploading_embedding(params, x):
    data_reuploading_ansatz(params[0:15])
    embedding.QuantumEmbedding1(x)

@qml.qnode(dev_fake)
def data_reuploading_QCNN_classifier(params, x, data):
    data_reuploading_embedding(params, x)
    embedding.Noisy_QCNN_eight(params[15: 15 + 8], ansatz)
    return qml.expval(qml.PauliZ(4))

@qml.qnode(dev_fake)
def QCNN_classifier(params, x, data):
    statepreparation(x, data)
    embedding.Noisy_QCNN_eight(params, ansatz)
    return qml.expval(qml.PauliZ(4))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost_NQE(weights, X_batch, Y_batch, data):
    preds = [QCNN_classifier(weights, x, data) for x in X_batch]
    return Linear_Loss(Y_batch, preds)

def cost_TQE(weights, X_batch, Y_batch, data):
    preds = [data_reuploading_QCNN_classifier(weights, x, data) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, data):

    num_weights_TQE = 15 + 8
    num_weights_NQE = 8
    
    init_weights = np.random.random(num_weights_TQE, requires_grad = True)
    weights_TQE = init_weights.reshape(23,1)
    weights_NQE = init_weights[15:23].reshape(8,1)
    opt_TQE = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    opt_NQE = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history_TQE, loss_history_NQE = [], []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights_NQE, cost_new_NQE = opt_NQE.step_and_cost(lambda v: cost_NQE(v, X_batch, Y_batch, data),
                                                     weights_NQE)
        weights_TQE, cost_new_TQE = opt_TQE.step_and_cost(lambda v: cost_TQE(v, X_batch, Y_batch, data),
                                                     weights_TQE)
        loss_history_NQE.append(cost_new_NQE)
        loss_history_TQE.append(cost_new_TQE)
        if it % 5 == 0:
            print("--------------------------------------")
            print("iteration: ", it, " cost_NQE: ", cost_new_NQE)
            print("iteration: ", it, " cost_TQE: ", cost_new_TQE)
            
    return loss_history_NQE, loss_history_TQE, weights_NQE, weights_TQE

In [5]:
feature_reduction = 'PCA8'
classes = [0,1]
data_type = 'mnist'
X_train, X_test, Y_train, Y_test = data.data_load_and_process(data_type, feature_reduction=feature_reduction, classes=classes)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [6]:


loss_history_NQE, loss_history_TQE, weights_NQE, weights_TQE = circuit_training(X_train, Y_train, data_type)
f = open('weights and losses mnist.txt', 'a')
f.write(f'Loss History NQE:')
f.write('\n')
f.write(str(loss_history_NQE))
f.write('\n')
f.write(f'Loss History TQE:')
f.write('\n')
f.write(str(loss_history_TQE))
f.write('\n')
f.write(f'Weights NQE:')
f.write('\n')
f.write(str(weights_NQE))
f.write('\n')
f.write(f'Weights TQE:')
f.write('\n')
f.write(str(weights_TQE))
f.write('\n')
f.close()

--------------------------------------
iteration:  0  cost_NQE:  0.5062905317769132
iteration:  0  cost_TQE:  0.4982490272373541
--------------------------------------
iteration:  5  cost_NQE:  0.5051880674448768
iteration:  5  cost_TQE:  0.4881971465629054
--------------------------------------
iteration:  10  cost_NQE:  0.4690012970168612
iteration:  10  cost_TQE:  0.5134889753566797
--------------------------------------
iteration:  15  cost_NQE:  0.4786640726329442
iteration:  15  cost_TQE:  0.48988326848249025
--------------------------------------
iteration:  20  cost_NQE:  0.46407263294422824
iteration:  20  cost_TQE:  0.490272373540856
--------------------------------------
iteration:  25  cost_NQE:  0.4210765239948118
iteration:  25  cost_TQE:  0.5026588845654995
--------------------------------------
iteration:  30  cost_NQE:  0.42107652399481194
iteration:  30  cost_TQE:  0.5255512321660182
--------------------------------------
iteration:  35  cost_NQE:  0.39987029831387805

In [7]:
def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)

prediction_NQE = [QCNN_classifier(weights_NQE, x, data_type) for x in X_test]
prediction_TQE = [data_reuploading_QCNN_classifier(weights_TQE, x, data_type) for x in X_test]


accuracy_NQE = accuracy_test(prediction_NQE, Y_test)
accuracy_TQE = accuracy_test(prediction_TQE, Y_test)

print(f'NQE Accuracy: {accuracy_NQE}')
print(f'TQE Accuracy: {accuracy_TQE}')


NQE Accuracy: 0.948936170212766
TQE Accuracy: 0.6151300236406619


In [8]:
feature_reduction = 'PCA8'
classes = [0,1]
data_type = 'fashion'
X_train, X_test, Y_train, Y_test = data.data_load_and_process(data_type, feature_reduction=feature_reduction, classes=classes)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [9]:
loss_history_NQE, loss_history_TQE, weights_NQE, weights_TQE = circuit_training(X_train, Y_train, data_type)
f = open('weights and losses fashion.txt', 'a')
f.write(f'Loss History NQE:')
f.write('\n')
f.write(str(loss_history_NQE))
f.write('\n')
f.write(f'Loss History TQE:')
f.write('\n')
f.write(str(loss_history_TQE))
f.write('\n')
f.write(f'Weights NQE:')
f.write('\n')
f.write(str(weights_NQE))
f.write('\n')
f.write(f'Weights TQE:')
f.write('\n')
f.write(str(weights_TQE))
f.write('\n')
f.close()

--------------------------------------
iteration:  0  cost_NQE:  0.4942282749675746
iteration:  0  cost_TQE:  0.49150453955901424
--------------------------------------
iteration:  5  cost_NQE:  0.48806744487678344
iteration:  5  cost_TQE:  0.522503242542153
--------------------------------------
iteration:  10  cost_NQE:  0.47879377431906617
iteration:  10  cost_TQE:  0.5238651102464331
--------------------------------------
iteration:  15  cost_NQE:  0.46115434500648517
iteration:  15  cost_TQE:  0.48184176394293127
--------------------------------------
iteration:  20  cost_NQE:  0.45123216601815824
iteration:  20  cost_TQE:  0.491958495460441
--------------------------------------
iteration:  25  cost_NQE:  0.44695201037613497
iteration:  25  cost_TQE:  0.48469520103761354
--------------------------------------
iteration:  30  cost_NQE:  0.4162127107652401
iteration:  30  cost_TQE:  0.4529831387808042
--------------------------------------
iteration:  35  cost_NQE:  0.3787937743190

In [10]:
prediction_NQE = [QCNN_classifier(weights_NQE, x, data_type) for x in X_test]
prediction_TQE = [data_reuploading_QCNN_classifier(weights_TQE, x, data_type) for x in X_test]


accuracy_NQE = accuracy_test(prediction_NQE, Y_test)
accuracy_TQE = accuracy_test(prediction_TQE, Y_test)

print(f'NQE Accuracy: {accuracy_NQE}')
print(f'TQE Accuracy: {accuracy_TQE}')

NQE Accuracy: 0.9105
TQE Accuracy: 0.669


In [11]:
feature_reduction = 'PCA8'
classes = [0,1]
data_type = 'mnist'
X_train, X_test, Y_train, Y_test = data.data_load_and_process(data_type, feature_reduction=feature_reduction, classes=classes)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

loss_history_NQE, loss_history_TQE, weights_NQE, weights_TQE = circuit_training(X_train, Y_train, data_type)
f = open('weights and losses mnist.txt', 'a')
f.write(f'Loss History NQE:')
f.write('\n')
f.write(str(loss_history_NQE))
f.write('\n')
f.write(f'Loss History TQE:')
f.write('\n')
f.write(str(loss_history_TQE))
f.write('\n')
f.write(f'Weights NQE:')
f.write('\n')
f.write(str(weights_NQE))
f.write('\n')
f.write(f'Weights TQE:')
f.write('\n')
f.write(str(weights_TQE))
f.write('\n')
f.close()

prediction_NQE = [QCNN_classifier(weights_NQE, x, data_type) for x in X_test]
prediction_TQE = [data_reuploading_QCNN_classifier(weights_TQE, x, data_type) for x in X_test]


accuracy_NQE = accuracy_test(prediction_NQE, Y_test)
accuracy_TQE = accuracy_test(prediction_TQE, Y_test)

print(f'NQE Accuracy: {accuracy_NQE}')
print(f'TQE Accuracy: {accuracy_TQE}')

--------------------------------------
iteration:  0  cost_NQE:  0.48573281452658884
iteration:  0  cost_TQE:  0.5065499351491569
--------------------------------------
iteration:  5  cost_NQE:  0.4803501945525292
iteration:  5  cost_TQE:  0.49280155642023343
--------------------------------------
iteration:  10  cost_NQE:  0.48138780804150455
iteration:  10  cost_TQE:  0.503242542153048
--------------------------------------
iteration:  15  cost_NQE:  0.471076523994812
iteration:  15  cost_TQE:  0.4849546044098573
--------------------------------------
iteration:  20  cost_NQE:  0.4557068741893645
iteration:  20  cost_TQE:  0.4995460440985733
--------------------------------------
iteration:  25  cost_NQE:  0.4283398184176395
iteration:  25  cost_TQE:  0.482230869001297
--------------------------------------
iteration:  30  cost_NQE:  0.4175097276264592
iteration:  30  cost_TQE:  0.4815823605706875
--------------------------------------
iteration:  35  cost_NQE:  0.39481193255512326
i

In [12]:
feature_reduction = 'PCA8'
classes = [0,1]
data_type = 'fashion'
X_train, X_test, Y_train, Y_test = data.data_load_and_process(data_type, feature_reduction=feature_reduction, classes=classes)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

loss_history_NQE, loss_history_TQE, weights_NQE, weights_TQE = circuit_training(X_train, Y_train, data_type)
f = open('weights and losses fashion.txt', 'a')
f.write(f'Loss History NQE:')
f.write('\n')
f.write(str(loss_history_NQE))
f.write('\n')
f.write(f'Loss History TQE:')
f.write('\n')
f.write(str(loss_history_TQE))
f.write('\n')
f.write(f'Weights NQE:')
f.write('\n')
f.write(str(weights_NQE))
f.write('\n')
f.write(f'Weights TQE:')
f.write('\n')
f.write(str(weights_TQE))
f.write('\n')
f.close()

prediction_NQE = [QCNN_classifier(weights_NQE, x, data_type) for x in X_test]
prediction_TQE = [data_reuploading_QCNN_classifier(weights_TQE, x, data_type) for x in X_test]


accuracy_NQE = accuracy_test(prediction_NQE, Y_test)
accuracy_TQE = accuracy_test(prediction_TQE, Y_test)

print(f'NQE Accuracy: {accuracy_NQE}')
print(f'TQE Accuracy: {accuracy_TQE}')

--------------------------------------
iteration:  0  cost_NQE:  0.48787289234760056
iteration:  0  cost_TQE:  0.49883268482490273
--------------------------------------
iteration:  5  cost_NQE:  0.46867704280155653
iteration:  5  cost_TQE:  0.509662775616083
--------------------------------------
iteration:  10  cost_NQE:  0.4596627756160831
iteration:  10  cost_TQE:  0.4752918287937744
--------------------------------------
iteration:  15  cost_NQE:  0.447016861219196
iteration:  15  cost_TQE:  0.5011024643320364
--------------------------------------
iteration:  20  cost_NQE:  0.4071335927367056
iteration:  20  cost_TQE:  0.47996108949416344
--------------------------------------
iteration:  25  cost_NQE:  0.4018158236057069
iteration:  25  cost_TQE:  0.5
--------------------------------------
iteration:  30  cost_NQE:  0.4057717250324254
iteration:  30  cost_TQE:  0.5010376134889754
--------------------------------------
iteration:  35  cost_NQE:  0.3702983138780803
iteration:  35 